In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 1: Network Access
-- Grants outbound HTTPS to NOAA (rainfall data) and USDA RMA (grid shapefile).
-- Run once per environment. Safe to re-run.
-- ═══════════════════════════════════════════════════════════

CREATE OR REPLACE NETWORK RULE noaa_cpc_rule
    MODE = EGRESS
    TYPE = HOST_PORT
    VALUE_LIST = ('ftp.cpc.ncep.noaa.gov:443', 'ftp.cpc.ncep.noaa.gov:80');

CREATE OR REPLACE NETWORK RULE rma_ftp_rule
    MODE = EGRESS
    TYPE = HOST_PORT
    VALUE_LIST = ('pubfs-rma.fpac.usda.gov:443', 'pubfs-rma.fpac.usda.gov:80');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION noaa_cpc_access
    ALLOWED_NETWORK_RULES = (noaa_cpc_rule, rma_ftp_rule)
    ENABLED = TRUE;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 2: Create Tables
-- PRF_RAINFALL_REALTIME stores daily CPC precipitation.
-- PRF_GRID_REFERENCE stores RMA grid centroids (loaded in Cell 4).
-- PRF_GRID_NORMALS is derived (built in Cell 7).
-- Safe to re-run — IF NOT EXISTS.
-- ═══════════════════════════════════════════════════════════

CREATE TABLE IF NOT EXISTS PRF_RAINFALL_REALTIME (
    observation_date    DATE            NOT NULL,
    latitude            FLOAT           NOT NULL,
    longitude           FLOAT           NOT NULL,
    precip_mm           FLOAT,
    precip_in           FLOAT,
    gauge_count         INT,
    file_type           VARCHAR(10),
    ingested_at         TIMESTAMP_NTZ,
    CONSTRAINT pk_prf_rain PRIMARY KEY (observation_date, latitude, longitude)
);

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 3: Grid Reference Ingestion Procedure
-- Downloads the official RMA PRF grid shapefile and computes
-- centroid lat/lon for each of the ~25,000 insurance grids.
-- Run once, or re-run when RMA updates the shapefile.
-- ═══════════════════════════════════════════════════════════

CREATE OR REPLACE PROCEDURE ingest_prf_grid_reference()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python', 'requests', 'geopandas', 'pandas', 'fiona', 'pyproj', 'shapely')
EXTERNAL_ACCESS_INTEGRATIONS = (NOAA_CPC_ACCESS)
HANDLER = 'run'
AS
$$
def run(session):
    import requests, zipfile, io, tempfile, os
    import geopandas as gpd
    import pandas as pd

    url = "https://pubfs-rma.fpac.usda.gov/pub/Miscellaneous_Files/VI_RI_Data/rainfall_index_grids.zip"
    r = requests.get(url)
    if r.status_code != 200:
        return f"Download failed: HTTP {r.status_code}"

    tmpdir = tempfile.mkdtemp()
    zipfile.ZipFile(io.BytesIO(r.content)).extractall(tmpdir)

    shp_path = None
    for root, dirs, files in os.walk(tmpdir):
        for f in files:
            if f.endswith('.shp'):
                shp_path = os.path.join(root, f)
                break

    if not shp_path:
        return "No .shp file found in archive"

    gdf = gpd.read_file(shp_path)
    if gdf.crs and gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    gdf['CENTER_LAT'] = gdf.geometry.centroid.y.round(6)
    gdf['CENTER_LON'] = gdf.geometry.centroid.x.round(6)
    gdf['MIN_LAT']    = gdf.geometry.bounds['miny'].round(6)
    gdf['MAX_LAT']    = gdf.geometry.bounds['maxy'].round(6)
    gdf['MIN_LON']    = gdf.geometry.bounds['minx'].round(6)
    gdf['MAX_LON']    = gdf.geometry.bounds['maxx'].round(6)

    df = pd.DataFrame(gdf.drop(columns='geometry'))
    df.columns = [c.upper() for c in df.columns]
    df['INGESTED_AT'] = pd.Timestamp.now()

    session.create_dataframe(df).write.mode("overwrite").save_as_table("PRF_GRID_REFERENCE")

    return (
        f"PRF_GRID_REFERENCE loaded: {len(df):,} grids | "
        f"Lat: {df['CENTER_LAT'].min():.3f} to {df['CENTER_LAT'].max():.3f} | "
        f"Lon: {df['CENTER_LON'].min():.3f} to {df['CENTER_LON'].max():.3f}"
    )
$$;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 4: Load Grid Reference
-- One-time load. Takes ~30 seconds.
-- ═══════════════════════════════════════════════════════════

CALL ingest_prf_grid_reference();

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 5: CPC Rainfall Ingestion Procedure
-- Downloads daily binary files from NOAA CPC, parses the grid,
-- converts units, and loads into PRF_RAINFALL_REALTIME.
--
-- Handles both historical backfill and daily loads.
-- Prefers UPDATED (quality-controlled) files, falls back to RT.
-- Includes grid format validation to catch NOAA spec changes.
--
-- NOAA CPC grid spec (from .ctl control file):
--   Source: ftp.cpc.ncep.noaa.gov/precip/CPC_UNI_PRCP/GAUGE_CONUS/DOCU/
--   120 rows × 300 cols, 0.25° step
--   Origin: 20.125°N, 129.875°W
--   Units: tenths of mm (0.1mm) — divide by 254 for inches
-- ═══════════════════════════════════════════════════════════

CREATE OR REPLACE PROCEDURE backfill_cpc_rainfall(start_date STRING, end_date STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python', 'requests', 'numpy', 'pandas')
EXTERNAL_ACCESS_INTEGRATIONS = (NOAA_CPC_ACCESS)
HANDLER = 'run'
AS
$$
def run(session, start_date, end_date):
    import requests
    import numpy as np
    import pandas as pd
    from datetime import datetime, timedelta

    # ── Grid constants (from NOAA .ctl control file) ─────────
    NROWS, NCOLS = 120, 300
    LAT_START, LON_START, STEP = 20.125, -129.875, 0.25
    GRID_FLOATS = NROWS * NCOLS    # 36,000 cells
    BASE = "https://ftp.cpc.ncep.noaa.gov/precip/CPC_UNI_PRCP/GAUGE_CONUS"

    current = datetime.strptime(start_date, '%Y%m%d')
    end     = datetime.strptime(end_date,   '%Y%m%d')

    results = []
    total_ok = 0
    total_skip = 0

    while current <= end:
        dt   = current.strftime('%Y%m%d')
        year = current.strftime('%Y')
        prev = str(int(year) - 1)
        obs  = f"{dt[:4]}-{dt[4:6]}-{dt[6:8]}"

        # ── Download: prefer UPDATED, fall back to RT ────────
        url_candidates = [
            (f"{BASE}/UPDATED/{year}/PRCP_CU_GAUGE_V1.0CONUS_0.25deg.lnx.{dt}.UPDATED", 'UPDATED'),
            (f"{BASE}/RT/{year}/PRCP_CU_GAUGE_V1.0CONUS_0.25deg.lnx.{dt}.RT", 'RT'),
            (f"{BASE}/RT/{prev}/PRCP_CU_GAUGE_V1.0CONUS_0.25deg.lnx.{dt}.RT", 'RT'),
        ]

        raw = None
        source = None
        for url, src in url_candidates:
            try:
                r = requests.get(url, timeout=30)
                if r.status_code == 200 and len(r.content) >= GRID_FLOATS * 4:
                    raw = r.content
                    source = src
                    break
            except:
                continue

        if raw is None:
            results.append(f"SKIP {dt}: no file found")
            total_skip += 1
            current += timedelta(days=1)
            continue

        # ── Grid format validation ───────────────────────────
        # Expected: 144,000 bytes (precip only) or 288,000 (precip + gauge count).
        # If NOAA changes the grid, file size changes and we reject
        # instead of silently loading garbage.
        expected_one = GRID_FLOATS * 4       # 144,000
        expected_two = GRID_FLOATS * 8       # 288,000
        actual_size  = len(raw)

        if actual_size not in (expected_one, expected_two):
            results.append(
                f"SKIP {dt}: GRID FORMAT CHECK FAILED — "
                f"expected {expected_one:,} or {expected_two:,} bytes, "
                f"got {actual_size:,}. Verify NOAA .ctl spec."
            )
            total_skip += 1
            current += timedelta(days=1)
            continue

        try:
            # ── Parse binary grid ────────────────────────────
            precip = np.frombuffer(raw[:GRID_FLOATS*4], dtype='<f4').reshape(NROWS, NCOLS)

            gauge = None
            if actual_size == expected_two:
                try:
                    gauge = np.frombuffer(raw[GRID_FLOATS*4:GRID_FLOATS*8], dtype='<f4').reshape(NROWS, NCOLS)
                except:
                    gauge = None

            # ── Convert to rows ──────────────────────────────
            rows = []
            for r_i in range(NROWS):
                for c_i in range(NCOLS):
                    val = float(precip[r_i, c_i])
                    if val >= 0:  # negative = no data (ocean/outside CONUS)
                        gc = None
                        if gauge is not None:
                            try:
                                gc = int(gauge[r_i, c_i])
                            except:
                                gc = None

                        # ── UNIT CONVERSION ──────────────────
                        # NOAA CPC stores as tenths of mm (0.1mm).
                        # val=110 means 11.0 mm = 0.433 inches.
                        # ─────────────────────────────────────
                        rows.append({
                            'OBSERVATION_DATE': obs,
                            'LATITUDE':   round(LAT_START + r_i * STEP, 3),
                            'LONGITUDE':  round(LON_START + c_i * STEP, 3),
                            'PRECIP_MM':  round(val / 10, 2),
                            'PRECIP_IN':  round(val / 254, 4),
                            'GAUGE_COUNT': gc,
                            'FILE_TYPE':  source,
                            'INGESTED_AT': pd.Timestamp.now()
                        })

            df = pd.DataFrame(rows)

            # ── Idempotent load (delete + append) ────────────
            session.sql(f"DELETE FROM PRF_RAINFALL_REALTIME WHERE observation_date = '{obs}'").collect()
            session.create_dataframe(df).write.mode("append").save_as_table("PRF_RAINFALL_REALTIME")

            rain_cells = len(df[df['PRECIP_MM'] > 0])
            results.append(f"OK {dt}: {len(df):,} cells, {rain_cells:,} with rain [{source}]")
            total_ok += 1

        except Exception as e:
            results.append(f"ERR {dt}: {str(e)[:100]}")
            total_skip += 1

        current += timedelta(days=1)

    summary = f"DONE: {total_ok} days loaded, {total_skip} skipped"
    results.insert(0, summary)
    return "\n".join(results)
$$;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 6A: Historical Backfill
-- Load Jan-Feb for 2023-2025 (needed for normals calibration)
-- plus 2026 YTD. Takes ~5 min per year.
-- Only run once, or after a table rebuild.
-- ═══════════════════════════════════════════════════════════

-- 2023 Jan-Feb
CALL backfill_cpc_rainfall('20230101', '20230228');
-- 2024 Jan-Feb
CALL backfill_cpc_rainfall('20240101', '20240229');
-- 2025 Jan-Feb
CALL backfill_cpc_rainfall('20250101', '20250228');
-- 2026 YTD (adjust end date as needed)
CALL backfill_cpc_rainfall('20260101', '20260213');

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 6B: Daily Load
-- Run this for a single day or small range.
-- Same procedure, just a narrow date window.
-- ═══════════════════════════════════════════════════════════

-- Yesterday (adjust date)
CALL backfill_cpc_rainfall('20260213', '20260213');

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 7: Build Implied Normals
-- Reverse-engineers the historical rainfall normal per grid:
--   Normal = Rain / (Index / 100)
-- Averages across 2023-2025 for stability.
-- CV% measures consistency — lower = more reliable.
-- Re-run after any rainfall data reload.
-- ═══════════════════════════════════════════════════════════

CREATE OR REPLACE TABLE PRF_GRID_NORMALS AS
WITH yearly_normals AS (
    SELECT 
        g.GRIDCODE AS grid_id,
        ri.YEAR,
        ri.INTERVAL_CODE,
        ri.INTERVAL_NAME,
        ri.INDEX_VALUE,
        ROUND(SUM(r.PRECIP_IN), 4) AS total_rain_in
    FROM PRF_RAINFALL_REALTIME r
    JOIN PRF_GRID_REFERENCE g
        ON ROUND(r.LATITUDE, 3) = ROUND(g.CENTER_LAT, 3)
        AND ROUND(r.LONGITUDE, 3) = ROUND(g.CENTER_LON, 3)
    JOIN RAIN_INDEX_PLATINUM_ENHANCED ri
        ON ri.GRID_ID = g.GRIDCODE 
        AND ri.INTERVAL_CODE = '625'
        AND ri.YEAR = YEAR(r.OBSERVATION_DATE)
    WHERE (r.OBSERVATION_DATE BETWEEN '2023-01-01' AND '2023-02-28' AND ri.YEAR = 2023)
       OR (r.OBSERVATION_DATE BETWEEN '2024-01-01' AND '2024-02-29' AND ri.YEAR = 2024)
       OR (r.OBSERVATION_DATE BETWEEN '2025-01-01' AND '2025-02-28' AND ri.YEAR = 2025)
    GROUP BY 1, 2, 3, 4, 5
),
with_normals AS (
    SELECT 
        grid_id, YEAR, INTERVAL_CODE, INTERVAL_NAME, INDEX_VALUE,
        total_rain_in,
        ROUND(total_rain_in / NULLIF(INDEX_VALUE / 100, 0), 4) AS implied_normal_in
    FROM yearly_normals
    WHERE INDEX_VALUE > 0
)
SELECT 
    grid_id,
    interval_code,
    interval_name,
    COUNT(*) AS years_used,
    ROUND(AVG(implied_normal_in), 4) AS normal_in,
    ROUND(STDDEV(implied_normal_in), 4) AS normal_stddev,
    ROUND(STDDEV(implied_normal_in) / NULLIF(AVG(implied_normal_in), 0) * 100, 1) AS cv_pct,
    CASE 
        WHEN STDDEV(implied_normal_in) / NULLIF(AVG(implied_normal_in), 0) < 0.05 THEN 'HIGH'
        WHEN STDDEV(implied_normal_in) / NULLIF(AVG(implied_normal_in), 0) < 0.15 THEN 'MEDIUM'
        ELSE 'LOW'
    END AS confidence_tier,
    CURRENT_TIMESTAMP() AS created_at
FROM with_normals
WHERE implied_normal_in > 0
GROUP BY 1, 2, 3;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 8: Data Summary Audit
-- Quick check: how many days loaded, by year and source type.
-- ═══════════════════════════════════════════════════════════

SELECT 
    YEAR(observation_date)                          AS year,
    file_type,
    COUNT(DISTINCT observation_date)                AS days_loaded,
    MIN(observation_date)                           AS first_day,
    MAX(observation_date)                           AS last_day,
    COUNT(*)                                        AS total_cells,
    ROUND(AVG(precip_in), 4)                        AS avg_precip_in,
    ROUND(MAX(precip_in), 4)                        AS max_precip_in
FROM PRF_RAINFALL_REALTIME
GROUP BY 1, 2
ORDER BY 1, 2;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 9: Spot Check — Known Locations
-- Sanity check: do these values look like real daily rainfall?
-- Kenedy TX should be dry (0-0.5"), East TX wetter.
-- ═══════════════════════════════════════════════════════════

SELECT 
    CASE 
        WHEN latitude = 27.375  AND longitude = -97.875  THEN 'King Ranch, TX'
        WHEN latitude = 35.125  AND longitude = -101.875 THEN 'Amarillo, TX'
        WHEN latitude = 30.625  AND longitude = -98.625  THEN 'Hill Country (Llano)'
        WHEN latitude = 31.625  AND longitude = -94.625  THEN 'East TX (Nacogdoches)'
        WHEN latitude = 26.875  AND longitude = -97.625  THEN 'Kenedy County, TX'
    END AS location_name,
    observation_date,
    precip_in,
    gauge_count,
    file_type
FROM PRF_RAINFALL_REALTIME
WHERE observation_date = (SELECT MAX(observation_date) FROM PRF_RAINFALL_REALTIME)
  AND (latitude, longitude) IN (
    (27.375, -97.875),
    (35.125, -101.875),
    (30.625, -98.625),
    (31.625, -94.625),
    (26.875, -97.625)
  )
ORDER BY location_name;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 10: Normals Sanity Check
-- Verify normals are physically reasonable.
-- Jan-Feb in South TX should be ~2-4". Panhandle ~1-2".
-- If you see values above 10", something is wrong.
-- ═══════════════════════════════════════════════════════════

SELECT 
    confidence_tier,
    COUNT(*)                        AS grids,
    ROUND(AVG(normal_in), 2)        AS avg_normal_in,
    ROUND(MIN(normal_in), 2)        AS min_normal_in,
    ROUND(MAX(normal_in), 2)        AS max_normal_in,
    ROUND(AVG(cv_pct), 1)           AS avg_cv_pct
FROM PRF_GRID_NORMALS
GROUP BY 1
ORDER BY 1;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 11: Prediction Accuracy — Grid Alignment Proof
-- Uses 2024 implied normals to predict 2025 index,
-- then compares to what RMA actually published.
-- This validates that our CPC→Grid coordinate matching is correct.
-- Median error < 1 point = grid alignment is solid.
-- ═══════════════════════════════════════════════════════════

WITH rainfall_2024 AS (
    SELECT g.GRIDCODE AS grid_id, ROUND(SUM(r.PRECIP_IN), 4) AS total_in
    FROM PRF_RAINFALL_REALTIME r
    JOIN PRF_GRID_REFERENCE g
        ON ROUND(r.LATITUDE, 3) = ROUND(g.CENTER_LAT, 3)
        AND ROUND(r.LONGITUDE, 3) = ROUND(g.CENTER_LON, 3)
    WHERE r.OBSERVATION_DATE BETWEEN '2024-01-01' AND '2024-02-29'
    GROUP BY 1
),
rainfall_2025 AS (
    SELECT g.GRIDCODE AS grid_id, ROUND(SUM(r.PRECIP_IN), 4) AS total_in
    FROM PRF_RAINFALL_REALTIME r
    JOIN PRF_GRID_REFERENCE g
        ON ROUND(r.LATITUDE, 3) = ROUND(g.CENTER_LAT, 3)
        AND ROUND(r.LONGITUDE, 3) = ROUND(g.CENTER_LON, 3)
    WHERE r.OBSERVATION_DATE BETWEEN '2025-01-01' AND '2025-02-28'
    GROUP BY 1
),
index_2024 AS (
    SELECT GRID_ID, INDEX_VALUE FROM RAIN_INDEX_PLATINUM_ENHANCED
    WHERE INTERVAL_CODE = '625' AND YEAR = 2024
),
index_2025 AS (
    SELECT GRID_ID, INDEX_VALUE FROM RAIN_INDEX_PLATINUM_ENHANCED
    WHERE INTERVAL_CODE = '625' AND YEAR = 2025
),
tx_grids AS (
    SELECT DISTINCT TRY_TO_NUMBER(SUB_COUNTY_CODE) AS grid_id
    FROM MAP_YTD
    WHERE INSURANCE_PLAN_CODE = '13' AND INTERVAL_CODE = '625'
      AND STATE_CODE = '48' AND DELETED_DATE IS NULL
),
comparisons AS (
    SELECT 
        r24.grid_id,
        ROUND((r25.total_in / NULLIF(r24.total_in / NULLIF(i24.INDEX_VALUE / 100, 0), 0)) * 100, 1) AS predicted,
        i25.INDEX_VALUE AS actual,
        ABS(ROUND(
            ROUND((r25.total_in / NULLIF(r24.total_in / NULLIF(i24.INDEX_VALUE / 100, 0), 0)) * 100, 1)
            - i25.INDEX_VALUE, 1)) AS abs_diff
    FROM rainfall_2024 r24
    JOIN rainfall_2025 r25 ON r24.grid_id = r25.grid_id
    JOIN index_2024 i24    ON r24.grid_id = i24.GRID_ID
    JOIN index_2025 i25    ON r24.grid_id = i25.GRID_ID
    JOIN tx_grids t        ON r24.grid_id = t.grid_id
    WHERE i24.INDEX_VALUE > 0
)
SELECT
    COUNT(*)                                                                        AS total_grids,
    ROUND(AVG(abs_diff), 2)                                                         AS avg_error,
    ROUND(MEDIAN(abs_diff), 2)                                                      AS median_error,
    SUM(CASE WHEN abs_diff <= 1 THEN 1 ELSE 0 END)                                 AS within_1pt,
    ROUND(SUM(CASE WHEN abs_diff <= 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1)   AS pct_within_1,
    SUM(CASE WHEN abs_diff <= 5 THEN 1 ELSE 0 END)                                 AS within_5pt,
    ROUND(SUM(CASE WHEN abs_diff <= 5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1)   AS pct_within_5,
    SUM(CASE WHEN abs_diff > 10 THEN 1 ELSE 0 END)                                 AS outliers_gt_10
FROM comparisons;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 12: Kenedy County Grid Check
-- Quick validation on the grids we watch most closely.
-- Normal should be ~2.8", not ~28".
-- ═══════════════════════════════════════════════════════════

SELECT 
    n.grid_id,
    ROUND(n.normal_in, 4)   AS normal_in,
    n.cv_pct,
    n.confidence_tier,
    g.CENTER_LAT,
    g.CENTER_LON
FROM PRF_GRID_NORMALS n
JOIN PRF_GRID_REFERENCE g ON g.GRIDCODE = n.GRID_ID
WHERE n.GRID_ID IN (8230, 8231)
ORDER BY n.GRID_ID;

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 13: Schedule Daily Load
-- Creates a Snowflake Task that runs backfill_cpc_rainfall
-- every day at 8 AM UTC (2 AM CST) for yesterday's data.
-- NOAA RT files are typically available by ~6 AM UTC.
--
-- To enable:  ALTER TASK daily_cpc_rainfall_load RESUME;
-- To pause:   ALTER TASK daily_cpc_rainfall_load SUSPEND;
-- To check:   SHOW TASKS LIKE 'daily_cpc%';
-- ═══════════════════════════════════════════════════════════

CREATE OR REPLACE TASK daily_cpc_rainfall_load
    SCHEDULE = 'USING CRON 0 8 * * * UTC'
    ALLOW_OVERLAPPING_EXECUTION = FALSE
    COMMENT = 'Daily CPC precipitation load for PRF tracker'
AS
    CALL backfill_cpc_rainfall(
        TO_CHAR(DATEADD('day', -1, CURRENT_DATE()), 'YYYYMMDD'),
        TO_CHAR(DATEADD('day', -1, CURRENT_DATE()), 'YYYYMMDD')
    );

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 14: Enable the Daily Task
-- Tasks are created in SUSPENDED state. This activates it.
-- ═══════════════════════════════════════════════════════════

ALTER TASK daily_cpc_rainfall_load RESUME;

-- Verify it's running
SHOW TASKS LIKE 'daily_cpc%';

In [ ]:
-- ═══════════════════════════════════════════════════════════
-- CELL 15: Task Run History
-- Check if the daily task is executing successfully.
-- ═══════════════════════════════════════════════════════════

SELECT 
    NAME,
    STATE,
    SCHEDULED_TIME,
    COMPLETED_TIME,
    RETURN_VALUE,
    ERROR_MESSAGE
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
    TASK_NAME => 'DAILY_CPC_RAINFALL_LOAD',
    RESULT_LIMIT => 10
))
ORDER BY SCHEDULED_TIME DESC;